# 3. Working With Pinecone Vector Database
Although Vector Databases have existed long before Large Language Models, vector DBs have become an important part of many LLM solutions. In particular, Retreival Augmented Generation (or RAG) architecture addresses LLM's halucinations and issues with longer-term memory by augmenting the user's prompt with the results of a search accross a vector DB. [Pinecone](https://www.pinecone.io/) is a cloud-based vector database that is easy to integrate with your CML workflow, as this notebook shows. 

Recall that in the previous exervice you created CML jobs to scrape a site and load each page into Pinecone vector DB. This notebook will focus on interacting with Pinecone from a Jupyter notebook.

![Exercise 3 overview](../assets/exercise_3.png)

### 3.1 Imports and global vars

In [1]:
import os
import pinecone
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import json
from openai import OpenAI

EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_REGION = os.getenv('PINECONE_REGION')

PINECONE_INDEX = os.getenv('PINECONE_INDEX')
dimension = 768

/home/cdsw/.local/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 3.2 Initialize Pinecone connection
Pinecone client is initialized with the parameters defined previously. 

In [2]:
print("initialising Pinecone connection...")
pc = Pinecone(api_key=PINECONE_API_KEY)
print("Pinecone initialised")

print(f"Getting '{PINECONE_INDEX}' as object...")
index = pc.Index(PINECONE_INDEX)
print("Success")

# Get latest statistics from index
current_collection_stats = index.describe_index_stats()
print('Total number of embeddings in Pinecone index is {}.'.format(current_collection_stats.get('total_vector_count')))

initialising Pinecone connection...
Pinecone initialised
Getting 'cml-index-se-west' as object...
Success
Total number of embeddings in Pinecone index is 15.


### 3.3 Function to peform the vector search 
The idea is to find a chunk from the Knowledge Base that is "close" to what the original user's prompt is. We perform a semantic search using the user's question, find the nearest knowledge base chunk, and return the content of that chunk along with its source and score.

In [3]:
base_url = "https://ai-inference.ainf-cdp.vayb-xokg.cloudera.site/namespaces/serving-default/endpoints/mistral-4b-rerank-a10g/v1"
model_name = "nvidia/nv-rerankqa-mistral-4b-v3"
# Load API key
OPENAI_API_KEY = json.load(open("/tmp/jwt"))["access_token"]

In [4]:
# Get embeddings for a user question and query Pinecone vector DB for nearest knowledge base chunk
def get_nearest_chunk_from_pinecone_vectordb(index, question):
    # Generate embedding for user question with embedding model
    retriever = SentenceTransformer(EMBEDDING_MODEL_REPO)
    xq = retriever.encode([question]).tolist()
    xc = index.query(vector=xq, top_k=5,
                 include_metadata=True)
    
    matching_files = []
    scores = []
    for i, match,  in enumerate(xc['matches']):
        # extract the 'file_path' within 'metadata'
        file_path = match['metadata']['file_path']
        print('file_path',file_path)
        print('chunch rank',i)
        # extract the individual scores for each vector
        score = match['score']
        scores.append(score)
        matching_files.append(file_path)

    # Return text of the nearest knowledge base chunk 
    # Note that this ONLY uses the first matching document for semantic search. matching_files holds the top results so you can increase this if desired.
    response = load_context_chunk_from_data(matching_files[0])
    sources = matching_files[0]
    score = scores[0]
    return response, sources, score

# Return the Knowledge Base doc based on Knowledge Base ID (relative file path)
def load_context_chunk_from_data(id_path):
    with open(id_path, "r") as f: # Open file in read mode
        return f.read()

### 3.4 Examine the results of the vector search
Given the text of the question, we can now perform a vector search and output the results in the notebook. An important detail here is the ability to interact with metadata (e.g. context source) which can be used to narrow down the search space and, more critically, for authorization. These approaches are out of scope of this lab. 

In [5]:
question = "What is CML?" ## (Swap with your own based on your dataset)3
context_chunk, sources, score = get_nearest_chunk_from_pinecone_vectordb(index, question)
print("\nContext Chunk: ")
print(context_chunk)
print("\nContext Source(s): ")
print(sources)
print("\nPinecone Score: ")
print(score)

file_path /home/cdsw/data/https:/docs.cloudera.com/machine-learning/cloud/product/topics/ml-product-overview.txt
chunch rank 0
file_path /home/cdsw/data/https:/docs.cloudera.com/machine-learning/cloud/architecture-overview/topics/ml-critical-and-noncritical-pods.txt
chunch rank 1
file_path /home/cdsw/data/https:/docs.cloudera.com/machine-learning/cloud/architecture-overview/topics/ml-architecture-overview-runtimes.txt
chunch rank 2
file_path /home/cdsw/data/https:/docs.cloudera.com/machine-learning/cloud/architecture-overview/topics/ml-architecture-overview1.txt
chunch rank 3
file_path /home/cdsw/data/https:/docs.cloudera.com/machine-learning/cloud/architecture-overview/topics/ml-architecture-overview-provisioning.txt
chunch rank 4

Context Chunk: 
What is CML?
Cloudera AI overviewCloudera Docs
Cloudera AI overview
Machine learning has become one of the most critical capabilities for modern businesses
 to grow and stay competitive today. From automating internal processes to optimizing

In [19]:
response = load_context_chunk_from_data(sources)

In [20]:
response

'ML RuntimesCloudera Docs ML Runtimes ML Runtimes are responsible for running data science workloads and intermediating       access to the underlying cluster.  Cloudera AI allows you to run any code via an interactive session, scheduled job, or deployed model          or application. Data Scientists can use interactive sessions to explore data, or develop a          model. They can create jobs and schedule them to run at specified times or productionize          their work as a model to provide a REST endpoint or as an application that offers an          interactive data dashboard for business users. All of these workloads run inside an ML Runtimes container on top of Kubernetes.  Cloudera ML Runtimes are purpose built to serve a specific use-case. They are available          with a single editor (for example, Workbench, Jupyterlab), ship a single language Kernel          (for example, Python 3.8 or R 4.0), and have a set of UNIX tools and utilities or language          libraries and 

### 3.5 Takeaways
* Vector search is a critical component of any LLM app using RAG architecture
* Cloudera's partner [Pinecone](https://www.pinecone.io/) provides a convenient SaaS offering for a Vector Database to support LLM RAG architecture
* Metadata from each entry in the Vector DB can be used to refine searches and add custom authorization frameworks

### Up Next: Go to Exercise 4
Where a gradio app is launched to complete the first iteration of the Q&A chat use case.